In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# Add the parent directory to the Python path
sys.path.append('..')

from src.data import load_data
from src.models import ModelWrapper
from src.optimizers import GeneticAlgorithmOptimizer
from src.utils import set_seeds, create_results_directory, extract_data_from_generator
from src.visualization import plot_predictions, plot_fitness_history, plot_time_series_predictions

# Set random seeds for reproducibility
set_seeds(42)

# Create a results directory
results_dir = create_results_directory()
print(f"Results will be saved in: {results_dir}")

In [ ]:
# Load the data using TimeseriesGenerator
train_generator, val_generator, test_generator, scaler_X, scaler_y = load_data()

# Print data sequences
print("Data sequences:")
print(f"Train sequences: {len(train_generator)}")
print(f"Validation sequences: {len(val_generator)}")
print(f"Test sequences: {len(test_generator)}")

# Get input shape from one batch
x_batch, y_batch = train_generator[0]
input_shape = x_batch.shape[1:]  # Exclude batch size dimension
print(f"Input shape: {input_shape}")
print(f"Output shape: {y_batch.shape}")

In [ ]:
# Define model architecture
layer_sizes = [256, 128, 64]

# Create the model
model_wrapper = ModelWrapper(input_shape, layer_sizes)
model_wrapper.model.summary()

# Compile the model
model_wrapper.compile(optimizer='sgd', loss='mse')

In [ ]:
# Define GA optimizer parameters
ga_optimizer_params = {
    "population_size": 20,
    "generations": 100,
    "tournament_size": 5,
    "mutation_rate": 0.01,
    "mutation_scale": 0.125,
    "patience": 25
}

# Instantiate the optimizer using the parameters
ga_optimizer = GeneticAlgorithmOptimizer(**ga_optimizer_params)

# Set the optimizer for the model
model_wrapper.set_optimizer(ga_optimizer)

# Train the model using the optimizer, passing results_dir
model_wrapper.train(train_generator, val_generator, results_dir=results_dir)

In [ ]:
# Evaluate the model
test_loss = model_wrapper.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")

# Generate predictions
test_X, test_y = extract_data_from_generator(test_generator)
y_pred_scaled = model_wrapper.predict(test_X)

# Inverse transform the predictions
test_predictions = scaler_y.inverse_transform(y_pred_scaled)
test_actual = scaler_y.inverse_transform(test_y)

# Flatten arrays
test_actual_flat = test_actual.flatten()
test_predictions_flat = test_predictions.flatten()

# Calculate evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(test_actual_flat, test_predictions_flat)
rmse = np.sqrt(mse)
r2 = r2_score(test_actual_flat, test_predictions_flat)

print("\nEvaluation Results:")
print(f"Test MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test R2 Score: {r2:.4f}")

# Save the model's weights
model_save_path = os.path.join(results_dir, 'best_model.weights.h5')
model_wrapper.model.save_weights(model_save_path)
print(f"Model weights saved to: {model_save_path}")

# Save fitness history to a text file
fitness_history_path = os.path.join(results_dir, 'fitness_history.txt')
with open(fitness_history_path, 'w') as f:
    for fitness in ga_optimizer.fitness_history:
        f.write(f"{fitness}\n")
print(f"Fitness history saved to: {fitness_history_path}")

# Plot predictions and save
plot_predictions(
    test_actual_flat,
    test_predictions_flat,
    "GA Model Predictions",
    save_path=os.path.join(results_dir, 'ga_predictions.png')
)

# Plot fitness history
fitness_history = ga_optimizer.fitness_history
plot_fitness_history(
    fitness_history,
    save_path=os.path.join(results_dir, 'ga_fitness_history.png')
)

# Time series plot of predicted vs actual power over time and save
plot_time_series_predictions(
    test_actual_flat,
    test_predictions_flat,
    title="GA Model Time Series Predictions",
    save_path=os.path.join(results_dir, 'ga_time_series_predictions.png')
)